In [ ]:
from google.colab import files
files.upload()
#upload the kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"princejohn","key":"abef5923911ae1721fef2358fe995e8e"}'}

In [ ]:
!mkdir -p ~/.kaggle  
!cp kaggle.json ~/.kaggle 
!chmod 600 ~/.kaggle/kaggle.json 

In [ ]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 22.7MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.9MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.14MB/s]


In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df=pd.read_csv("/content/train.csv",index_col="PassengerId")

In [ ]:
l,b=df.shape
df["Age"].fillna(df["Age"].median(),inplace=True)
df["Cabin"].fillna("ab",inplace=True)
df["Embarked"].fillna("S",inplace=True)

In [ ]:
df["Sex"]=df["Sex"].replace({"male":str(1),"female":str(0)}).astype(int)

In [ ]:
q=df["Fare"].quantile(0.99)
df2=df[df["Fare"]<q]
tar=df2['Survived']
fea=df2.drop(["Survived","Name","Ticket","Cabin"],axis=1)


In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(fea,tar,test_size=0.3,random_state=42,shuffle=True)

In [ ]:
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.metrics import confusion_matrix , accuracy_score
from xgboost import XGBClassifier
reg=XGBClassifier()


In [ ]:
params={"n_estimators":[67,70,100,120],'reg_lambda':[2,1],'gamma':[0,0.3,0.2,0.1]
       ,'eta':[0.06,0.05,0.04]
        ,"max_depth":[3,5],'objective':['binary:logistic']}

In [ ]:


clf=GridSearchCV(reg,params,cv=10,n_jobs=-1,verbose=1)
clf.fit(xtrain,ytrain)



Fitting 10 folds for each of 192 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 764 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 1764 tasks      | elapsed:   56.2s
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed:  1.0min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'eta': [0.06, 0.05, 0.04], 'gamma': [0, 0.3, 0.2, 0.1],
                         'max_depth': [3, 5],
                         'n_estimators': [67, 70, 1

In [ ]:
ypred=clf.predict(xtrain)
tpred=clf.predict(xtest)
print(accuracy_score(ypred,ytrain))
print(accuracy_score(tpred,ytest))
clf.best_params_

0.893030794165316
0.8452830188679246


{'eta': 0.06,
 'gamma': 0.3,
 'max_depth': 5,
 'n_estimators': 67,
 'objective': 'binary:logistic',
 'reg_lambda': 2}

In [ ]:
tes=pd.read_csv("/content/test.csv",index_col="PassengerId")

In [ ]:
tes["Age"]=tes["Age"].fillna(tes["Age"].median())
tes["Sex"]=le.fit_transform(tes["Sex"])
tes["Embarked"]=le.fit_transform(tes["Embarked"])
tes["Fare"]=tes["Fare"].fillna(tes["Fare"].median())
tes.isnull().sum()
tfea=tes.drop(["Name","Ticket","Cabin"],axis=1)
#tfea[["Fare","Age"]]=scaler.fit_transform(tfea[["Fare","Age"]])
my_pred=clf.predict(tfea)
submit=pd.DataFrame(tes.index,columns=["PassengerId"])
submit["Survived"]=my_pred
submit



,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
submit.to_csv('XGB_Classifier.csv',index=False)